# Build Deep Learning Module
Data is preprocessed and separated and ready for modelling.

Framework for our basic neural network:

- For input layer, add the number of input features equal to the number of variables in our feature DataFrame.
- For hidden layers, our deep learning model structure will be slightly different.
     - Add two hidden layers with only a few neurons in each layer.
     - To create the second hidden layer, add another Keras Dense class while defining our model.
     - All hidden layers will use the relu activation function to identify nonlinear characteristics from the input values.
- For output layer, use the same parameters from the basic neural network including the sigmoid activation function. The sigmoid activation function will helps predict the probability that an employee is at risk for attrition.

##### Generate a deep learning model that can help to identify whether or not a person is likely to depart from the company given his or her current employee profile.

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np
from scipy import stats

# Import our input dataset
attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


###### Preprocessing
There are multiple columns with categorical values. Generate a list of categorical variable names.

In [2]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()
attrition_cat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

##### 
There are 8 categorical variables that need encoding. Before looping through these variables and encode them using Scikit-learn’s OneHotEncoder, make sure that none of the categorical variables have more than 10 unique values and require bucketing.

In [3]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

##### 
According to the nunique method, none of the categorical variables have more than 10 unique values, which means we’re ready to encode.

Scikit-learn is flexible enough to perform all of the one-hot encodings at the same time. The only difference from our single variable example is we need to pass our attrition_cat variable list.

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse = False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


##### 
Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset.

In [5]:
attrition_df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

#####- 
Split into training and testing data, then standardize the numerical variables using Scikit-learn’s StandardScaler module.

NOTE: We need to split our training and testing data before fitting our StandardScaler instance. This prevents testing data from influencing the standardization function.

In [6]:
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


### Do not forget this part!

In [7]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [8]:
attrition_df.columns

Index(['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome',
       'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager', 'Attrition_No',
       'Attrition_Yes', 'BusinessTravel_Non-Travel',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'Department_Human Resources', 'Department_Research & Development',
       'Department_Sales', 'EducationField_Human Resources',
       'EducationField_Life Sciences', 'EducationField_Marketing',
       'EducationField_Medical', 'EducationField_Other',
       'EducationField_Technical Degree', 'Gen

### Now the Split

In [9]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes", "Attrition_No"], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 78)

### 
NOTE: For our purposes, we want to build a model that will predict whether or not a person is at risk for attrition; therefore, we must separate the “Attrition” columns from the rest of the input data. In fact, because the attrition data is dichotomous (one of two values), we only need to keep the “Attrition_Yes” column. We can ignore the “Attrition_No” column because it is redundant.

Now that training and testing data have been allocated, we’re ready to build our StandardScaler object and standardize the numerical features.

In [10]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Build Deep Learning Module
Data is preprocessed and separated and ready for modelling.

Framework for our basic neural network:

For input layer, add the number of input features equal to the number of variables in our feature DataFrame.
For hidden layers, our deep learning model structure will be slightly different.
Add two hidden layers with only a few neurons in each layer.
To create the second hidden layer, add another Keras Dense class while defining our model.
All hidden layers will use the relu activation function to identify nonlinear characteristics from the input values.
For output layer, use the same parameters from the basic neural network including the sigmoid activation function. The sigmoid activation function will helps predict the probability that an employee is at risk for attrition.

In [11]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, 
                             input_dim = number_input_features, 
                             activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, 
                             activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, 
                             activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 448       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


Time to compile our model and define the loss and accuracy metrics. Since we want to use our model as a binary classifier, we’ll use the binary_crossentropy loss function, adam optimizer, and accuracy metrics, which are the same parameters we used for our basic neural network.

In [12]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

### Train and evaluate the deep learning model

In [13]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs = 100)

Epoch 1/100
35/35 [==============================] - 0s 669us/step - loss: 433.1989 - accuracy: 0.5853
Epoch 2/100
35/35 [==============================] - 0s 663us/step - loss: 167.3794 - accuracy: 0.7296
Epoch 3/100
35/35 [==============================] - 0s 688us/step - loss: 63.7066 - accuracy: 0.6833
Epoch 4/100
35/35 [==============================] - 0s 686us/step - loss: 40.1964 - accuracy: 0.6969
Epoch 5/100
35/35 [==============================] - 0s 674us/step - loss: 28.8588 - accuracy: 0.6915
Epoch 6/100
35/35 [==============================] - 0s 718us/step - loss: 20.9551 - accuracy: 0.7015
Epoch 7/100
35/35 [==============================] - 0s 696us/step - loss: 12.4799 - accuracy: 0.7142
Epoch 8/100
35/35 [==============================] - 0s 719us/step - loss: 7.6443 - accuracy: 0.7350
Epoch 9/100
35/35 [==============================] - 0s 678us/step - loss: 5.8919 - accuracy: 0.7287
Epoch 10/100
35/35 [==============================] - 0s 672us/step - loss: 5.1370

35/35 [==============================] - 0s 907us/step - loss: 1.3622 - accuracy: 0.7804
Epoch 82/100
35/35 [==============================] - 0s 818us/step - loss: 0.9619 - accuracy: 0.7995
Epoch 83/100
35/35 [==============================] - 0s 766us/step - loss: 0.8978 - accuracy: 0.7740
Epoch 84/100
35/35 [==============================] - 0s 747us/step - loss: 0.6616 - accuracy: 0.8167
Epoch 85/100
35/35 [==============================] - 0s 700us/step - loss: 0.5733 - accuracy: 0.8330
Epoch 86/100
35/35 [==============================] - 0s 708us/step - loss: 0.9159 - accuracy: 0.7967
Epoch 87/100
35/35 [==============================] - 0s 711us/step - loss: 0.6090 - accuracy: 0.8149
Epoch 88/100
35/35 [==============================] - 0s 711us/step - loss: 0.5688 - accuracy: 0.8430
Epoch 89/100
35/35 [==============================] - 0s 664us/step - loss: 1.1756 - accuracy: 0.8004
Epoch 90/100
35/35 [==============================] - 0s 719us/step - loss: 0.8980 - accuracy: 

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 1.3758 - accuracy: 0.8641
Loss: 1.3757870197296143, Accuracy: 0.864130437374115


368/1 - 0s - loss: 0.4860 - accuracy: 0.8668
Loss: 0.7585730708163717, Accuracy: 0.866847813129425

TAKEAWAY: Looking at our deep learning model’s performance metrics, the model was able to correctly identify employees who are at risk of attrition approximately 87% of the time. Considering that our input data included more than 30 different variables with more than 1,400 data points, the deep learning model was able to produce a fairly reliable classifier.

### Implement checkpoints to our deep learning model

Compile and train our model using checkpoints. To use checkpoints, define the checkpoint file name and directory path. For our purposes, we’ll label our checkpoints by epoch number and contain them within their own folder to ensure that our checkpoint files are neat, organized, and easily identifiable.

In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

Create a `callback object` for our deep learning model. A callback object is used in the Keras module to define a set of functions that will be applied at specific stages of the training process.

There are a number of different callback functions available that can create log files, force training to stop, send training status messages, or in our case save model checkpoints.

Create an effective checkpoint callback using the ModelCheckpoint method using the following parameters:

- filepath = checkpoint_path — the checkpoint directory and file structure we defined previously
- verbose = 1 — we’ll be notified when a checkpoint is being saved to the directory
- save_weights_only = True — saving the full model each time can fill up a hard drive very quickly
  - this ensures that the checkpoint files take up minimal space
- save_freq = 1000 — checkpoints will be saved every thousand samples tested (across all epochs)

In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
35/35 [==============================] - 0s 2ms/step - loss: 0.6570 - accuracy: 0.6234

Epoch 00001: saving model to checkpoints/weights.01.hdf5
Epoch 2/100
35/35 [==============================] - 0s 2ms/step - loss: 0.5556 - accuracy: 0.6706

Epoch 00002: saving model to checkpoints/weights.02.hdf5
Epoch 3/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4995 - accuracy: 0.7269

Epoch 00003: saving model to checkpoints/weights.03.hdf5
Epoch 4/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4640 - accuracy: 0.7550

Epoch 00004: saving model to checkpoints/weights.04.hdf5
Epoch 5/100
35/35 [==============================] - 0s 902us/step - loss: 0.4398 - accuracy: 0.7868

Epoch 00005: saving model to checkpoints/weights.05.hdf5
Epoch 6/100
35/35 [==============================] - 0s 908us/step - loss: 0.4230 - accuracy: 0.8140

Epoch 00006: saving model to checkpoints/weights.06.hdf5
Epoch 7/100
35/35 [==============================] - 0

35/35 [==============================] - 0s 1ms/step - loss: 0.2190 - accuracy: 0.9147

Epoch 00053: saving model to checkpoints/weights.53.hdf5
Epoch 54/100
35/35 [==============================] - 0s 901us/step - loss: 0.2177 - accuracy: 0.9129

Epoch 00054: saving model to checkpoints/weights.54.hdf5
Epoch 55/100
35/35 [==============================] - 0s 831us/step - loss: 0.2161 - accuracy: 0.9147

Epoch 00055: saving model to checkpoints/weights.55.hdf5
Epoch 56/100
35/35 [==============================] - 0s 1ms/step - loss: 0.2142 - accuracy: 0.9138

Epoch 00056: saving model to checkpoints/weights.56.hdf5
Epoch 57/100
35/35 [==============================] - 0s 1ms/step - loss: 0.2125 - accuracy: 0.9138

Epoch 00057: saving model to checkpoints/weights.57.hdf5
Epoch 58/100
35/35 [==============================] - 0s 1000us/step - loss: 0.2110 - accuracy: 0.9156

Epoch 00058: saving model to checkpoints/weights.58.hdf5
Epoch 59/100
35/35 [==============================] - 0s 1

After running the previous code, we have created our trained model within the Python session, as well as a folder of checkpoints we can use to restore previous model weights. Now if we ever need to restore weights, we can use the Keras Sequential model’s load_weights method to restore the model weights.

To test this functionality, let’s define another deep learning model, but restore the weights using the checkpoints rather than training the model.

In [17]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3498 - accuracy: 0.8859
Loss: 0.3497791290283203, Accuracy: 0.885869562625885


In [18]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [19]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [20]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3498 - accuracy: 0.8859
Loss: 0.3497791290283203, Accuracy: 0.885869562625885
